# CPA - Key Recovering

When focusing on key recovering, CPA uses the Pearson Correlation Coefficient to reveal the "collision" of secret constant data with public variable information.

`_fixedkey` traces are used to reveal the fixed secret key.

In [1]:
import numpy as np
from cpa_key import poi_selectors, datasets
import securec
import securec.cpa
import securec.cpa_vis

In [10]:
poi_selector_name = 'subbytes'
poi_selector = poi_selectors[poi_selector_name]

In [11]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import CrosshairTool, Title
from bokeh.palettes import Category20_20

output_notebook()

Loading BokehJS ...

## Correlation vs time

In [12]:
p = figure(height=300, sizing_mode='stretch_width')
p.title = f'Correlation vs time on {poi_selector_name}.'
p.add_tools(CrosshairTool())

for i, (dataname, data) in enumerate(datasets.items()):
    pearsons = abs(securec.cpa.pearson_pointwise(data['trace'], poi_selector(data)))
    p.line(range(0, len(pearsons)), pearsons, line_color=Category20_20[i], legend_label=dataname)
show(p)


## Correlation vs number of samples

In [13]:
intermediates = {
    'addkey': lambda: np.array([data['input'][:, 0] ^ guess for guess in range(256)]),
    'hw_addkey': lambda: securec.cpa.intermediates_aes_sbox(data['input']),
    'subbytes': lambda: np.array([securec.cpa.sbox_vec(data['input'][:, 0] ^ guess) for guess in range(256)]),
    'hw_subbytes': lambda: securec.cpa.intermediates_aes_sbox_input(data['input']),
}[poi_selector_name]


for dataname, data in datasets.items():
    securec.cpa_vis.plot_traces_vs_correlation(
        data['trace'], 
        intermediates(),
        figure_title=f'Correlation vs samples on {dataname} {poi_selector_name}'
    )